# Points of Dispense - Anthrax Attack in Downtown Pittsburgh
#### Minimize maximum distance and vary number of PODs
Maxwell Kennady, Nora Murray, Elizabeth Speigle

In [1]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

### Optimization

Read in data from files

In [2]:
distances = pd.read_csv('data/OD_Pairs_Distances.csv')
population = pd.read_excel('data/BG_master.xlsx')

In [3]:
dist_miles = distances.pivot(index='block_group', columns='pod_id', values='Miles')
dist_time = distances.pivot(index='block_group', columns='pod_id', values='TravelTime')

In [4]:
dist = dist_miles.values
N = population['population'].values
prop = population['anthrax'].values

Create indices for block groups and PODs

In [5]:
blocks = range(len(N))
pods = range(len(dist[0]))

Initialize model for POD locations

In [6]:
pods_open_results = {}

In [7]:
pods_open = range(1, 40, 2)

In [8]:
for i in pods_open: 
    ### Initialize model for POD locations
    m = gp.Model('POD_locations')
    
    ### Add decision variables
    x = m.addVars(pods, vtype=GRB.BINARY, name='x')
    y = m.addVars(blocks, pods, vtype=GRB.BINARY, name='y')
    w = m.addVars(blocks, vtype=GRB.BINARY, name='w')
    
    ### Set objective to minimize maximum distance
    z = m.addVar(vtype=GRB.CONTINUOUS, name='z')
    m.setObjective(z, GRB.MINIMIZE)
    
    ### Add constraints
    #  y[i,j] can only be 1 if x[i] is also 1, meaning POD i is opened
    m.addConstrs((y[j, i] <= x[i] for i in pods for j in blocks), name='y_if_x')
    
    # each block group must be assigned one shelter
    m.addConstrs((gp.quicksum(y[j, i] for i in pods) == 1
             for j in blocks), name='all_blocks_assigned')
    
    # number of PODs opened must be less than num_pods
    m.addConstr((gp.quicksum(x[i] for i in pods) <= i), name='pods_opened')
    
    # w[j] should be 1 if prop[j] is not zero, and zero otherwise
    m.addConstrs((w[j] * 9999 >= prop[j] * N[j] for j in blocks), 
                 name='Prop_Above_Zero')
    
    # maximum distance
    m.addConstrs((z >= dist[j, i] * w[j] * y[j, i] for i in pods for j in blocks), 
             name='Distance_Below_Max')
    
    ### Optimize
    print('Solving for:', i, 'PODs')
    m.optimize()
    
    ### Analysis
    block_pod_list = [[j,i] for j in blocks for i in pods if y[j, i].x==1]
    pods_open_results[i] = block_pod_list

Using license file C:\Users\Elizabeth\gurobi.lic
Academic license - for non-commercial use only
Solving for: 1 PODs
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 53901 rows, 52848 columns and 156247 nonzeros
Model fingerprint: 0xe4cbac38
Model has 51700 quadratic constraints
Variable types: 1 continuous, 52847 integer (52847 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  QMatrix range    [8e-02, 4e+01]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+03]
Presolve added 3506 rows and 0 columns
Presolve removed 0 rows and 98 columns
Presolve time: 0.37s
Presolved: 151595 rows, 99844 columns, 399829 nonzeros
Variable types: 1 continuous, 99843 integer (99843 binary)
Found heuristic solution: objective 9.1898996

Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...


Root simplex log...

Iteration    Objective       Prima

In [9]:
pods_anthrax_minmax2 = open('pods_anthrax_minmax2','wb')
pickle.dump(pods_open_results, pods_anthrax_minmax2)
pods_anthrax_minmax2.close()